<a href="https://colab.research.google.com/github/leo-alves-melo/busPredictor/blob/master/dumpBusPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Creating a dump algorithm for bus prediction given the location of the vehicle

In [2]:
# Install dependencies
!pip install arrow

In [0]:
# Importing packages

from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
import pprint
import json
import pandas as pd
import numpy as np
import arrow
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
from functools import reduce
from math import sin, cos, sqrt, atan2, radians
from datetime import datetime
%matplotlib inline

In [0]:
# Setting variables

base_api = 'https://api.demo.konkerlabs.net'
username = "student_smartcampus@konkerlabs.com"
password = 'SdExnFiHcGrK'

circulino = "3b7c728c-cf13-45ca-a803-d94a598113d0"
circulino1 = "0b525e25-6962-4ee4-8f8a-834e3e33698d"
circulino2 = "8ab6ae21-9d9e-4637-8c56-60ba15691fd2"
circulino3 = "8f4ab4d2-203e-4043-b4b1-bffca39f8686"
circulino4 = "6ce968a1-a32b-4f8c-bc39-4463f50f4591"
circulino5 = "a028f083-8375-45ec-a42f-7e864fd5f8d7"
circulino6 = "eb0ec27f-16d0-4504-a331-af5b1e24eac6"
circulino7 = "826f5fe4-ec9b-4de7-87f8-93be40963612"
circulino8 = "8a33a2fe-8325-4c71-b937-8cc75b85bca4"

# These are the available circulars because the others do not have data
available_circulars = [circulino4, circulino5, circulino6, circulino7, circulino8]

# Coordinates of the home of the circular
latitude_home = -22.829636
longitude_home = -47.061038
home = {'geolocation':{'lat': latitude_home, 'lon': longitude_home}}

In [0]:
# Defining Functions

def get_data(guid):
    global oauth
    
    dt_start = arrow.utcnow().to('America/Sao_Paulo').floor('day')
    timestamp_init = dt_start.shift(days=-7).isoformat()
    channel = 'info'
    timestamp_final = arrow.utcnow().isoformat()
    application = 'default'
    result = []
    lastlen = 0
    while timestamp_init < timestamp_final:
        try:
            stats = oauth.get("https://api.demo.konkerlabs.net/v1/{}/incomingEvents?q=device:{} channel:{} timestamp:>{}&sort=oldest&limit=10000".format(application,guid, channel, timestamp_init)).json()['result']
            if (len(stats)<2) and (lastlen<20000):
                break
            timestamp_init = stats[-1]['timestamp']
            result.extend(stats)
            lastlen = len(stats)
        except:
            pass
    return result

# Calculates the distance in km
def distance_between(position1, position2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(position1['geolocation']['lat'])
    lon1 = radians(position1['geolocation']['lon'])
    lat2 = radians(position2['geolocation']['lat'])
    lon2 = radians(position2['geolocation']['lon'])

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    distance = R * c
    
    return distance

# Calculates the velocity in km/h
def velocity_between(position1, position2):
    
    distance = distance_between(position1, position2)
    
    time1 = datetime.strptime(position1['timestamp'], '%Y-%m-%dT%H:%M:%S.%fZ')
    time2 = datetime.strptime(position2['timestamp'], '%Y-%m-%dT%H:%M:%S.%fZ')

    time_delta = (time2 - time1).total_seconds()/3600

    return distance/time_delta

In [0]:
# Authenticating

client = BackendApplicationClient(client_id=username)
oauth = OAuth2Session(client=client)
token = oauth.fetch_token(token_url='{}/v1/oauth/token'.format(base_api),client_id=username,client_secret=password)

In [0]:
# Reading all circular data

data = []

for circular in available_circulars:
    data += get_data(circular)

In [0]:
# Getting all possible id_linha

id_lines = {}
for json in data:
    id_line = json['payload']['id_linha']
    if id_line in id_lines:
        id_lines[id_line] += 1
    else:
        id_lines[id_line] = 1

print(id_lines)

{2: 31214, 4: 91564, 3: 24007, 1: 88110}


In [0]:

    
print('')


In [0]:
# Separate my dataset by id_line and days

dataset = []

for id_line in id_lines:
    all_elements_of_this_id = list(filter(lambda json: json['payload']['id_linha'] == id_line, data))
    # Separate by day
    elements_separated_by_day = reduce(lambda elements, json: df, all_elements_of_this_id, {})

NameError: ignored